In [1]:
import pandas as pd
from pandas.io.json import json_normalize 
import os
import plotly.express as px
import json 

dataset = "CIFAR100"
all_subdirs = [os.path.join(dataset,d) for d in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, d)) and d != "cifar-100-python"]
print(all_subdirs)
latest_folder = max(all_subdirs, key=os.path.getmtime)

data = []
with open(os.path.join(latest_folder, "results.jsonl"), 'r', encoding='utf-8') as f:
    for line in f:
       data.append(json.loads(line))

df = json_normalize(data)
df = df.applymap(lambda x: x[0] if (isinstance(x, list) and len(x) == 1) else x)

def nice_name(row):
    if row["model"] == "GNCLClassifier":
        return u"GNLC \u03BB = {}".format(row["l_reg"]) 
    elif row["model"] == "BaggingClassifier":
        return "Bagging"
    elif row["model"] == "E2EEnsembleClassifier":
        return "E2E"
    elif row["model"] == "SKLearnModel":
        return "Single"
    else:
        return row["model"]

def base_type(row):
    if "binary" in row["base_estimator"]:
        mtype = "binary"
    else:
        mtype = "float"

    if "128" in row["base_estimator"]:
        size = "small"
    else:
        size = "large"

    return row["base_estimator"].split("_")[0] + " " + size + " " + mtype

print(df.columns)     
df["nice_name"] = df.apply(nice_name, axis=1)
df["model_type"] = df.apply(base_type, axis=1)
df["complexity"] = df.apply(lambda x : x["scores.params_test"] if "float" in x["model_type"] else x["scores.params_test"] / 32.0, axis=1)
df["scores.avg_accurcay_test"] = df["scores.avg_accurcay_test"]
df["scores.accuracy_test"] = 100.0*df["scores.accuracy_test"]
df = df.round(decimals = 3)

print("{} experiments done".format(len(df)))
cols = df.columns
cols = ["nice_name", "model_type", "mode", "scores.accuracy_test", "scores.params_test"] + list(set(cols) - set(["nice_name", "model_type", "mode", "scores.accuracy_test", "scores.params_test"]))
df = df.reindex(cols, axis=1)
df
#dfbinary = df[df["model_type"] == "binary"]
#dffloat = df[df["model_type"] == "float"]  
#dffloat


['CIFAR100/13-10-2020-22:48:28', 'CIFAR100/01-10-2020-20:52:08', 'CIFAR100/13-10-2020-22:33:43', 'CIFAR100/13-10-2020-22:38:46', 'CIFAR100/15-10-2020-22:29:04', 'CIFAR100/13-10-2020-22:44:11', 'CIFAR100/13-10-2020-23:46:30', 'CIFAR100/01-10-2020-18:03:38', 'CIFAR100/13-10-2020-23:40:44', 'CIFAR100/13-10-2020-23:27:01', 'CIFAR100/13-10-2020-22:40:15', 'CIFAR100/13-10-2020-22:34:54', 'CIFAR100/13-10-2020-23:43:43', 'CIFAR100/01-10-2020-17:55:43', 'CIFAR100/15-10-2020-22:21:39', 'CIFAR100/13-10-2020-23:48:31', 'CIFAR100/13-10-2020-23:48:03', 'CIFAR100/13-10-2020-19:07:51', 'CIFAR100/13-10-2020-22:37:16', 'CIFAR100/13-10-2020-23:49:55', 'CIFAR100/13-10-2020-23:42:29', 'CIFAR100/13-10-2020-23:03:19', 'CIFAR100/15-10-2020-22:18:16', 'CIFAR100/13-10-2020-22:08:02', 'CIFAR100/15-10-2020-22:19:24', 'CIFAR100/01-10-2020-18:00:28', 'CIFAR100/01-10-2020-20:26:11', 'CIFAR100/01-10-2020-20:57:47', 'CIFAR100/15-10-2020-22:04:07', 'CIFAR100/01-10-2020-19:01:55', 'CIFAR100/15-10-2020-22:20:11', 'CIFAR1

,nice_name,model_type,mode,scores.accuracy_test,scores.params_test,local_mode,scoring.diversity,scores.params_train,seed,no_runs,...,optimizer.batch_size,test,scores.diversity_train,scheduler.T_max,scores.avg_accurcay_train,model,out_path,transformer,scoring.loss,experiment_id
0,Single,mobilenetv3 large float,NaN,65.12,1610844,False,diversity,1610844,None,1,...,128,"[/data/s1/buschjae/CIFAR100/, True]",0.000,20,0.000,SKLearnModel,/home/buschjae/projects/bnn/CIFAR100/15-10-202...,Compose(\n Normalize(mean=[-2.4290657439446...,loss,0
1,E2E,mobilenetv3 large float,NaN,59.49,25773504,False,diversity,25773504,None,1,...,128,"[/data/s1/buschjae/CIFAR100/, True]",33.587,20,4.080,E2EEnsembleClassifier,/home/buschjae/projects/bnn/CIFAR100/15-10-202...,Compose(\n Normalize(mean=[-2.4290657439446...,loss,2
2,Bagging,mobilenetv3 large float,NaN,72.25,25773504,False,diversity,25773504,None,1,...,128,"[/data/s1/buschjae/CIFAR100/, True]",0.600,20,82.912,BaggingClassifier,/home/buschjae/projects/bnn/CIFAR100/15-10-202...,Compose(\n Normalize(mean=[-2.4290657439446...,loss,1
3,SMCLClassifier,mobilenetv3 large float,NaN,22.45,25773504,False,diversity,25773504,None,1,...,128,"[/data/s1/buschjae/CIFAR100/, True]",3.510,20,6.345,SMCLClassifier,/home/buschjae/projects/bnn/CIFAR100/15-10-202...,Compose(\n Normalize(mean=[-2.4290657439446...,loss,3


In [14]:
import numpy as np
import plotly.io as pio

if not os.path.exists("images"):
    os.mkdir("images") 

pio.orca.config.use_xvfb = True

mtype, n_estimators = "ResNet11 float", [4]
# mtype, n_estimators = "EfficientNetB0 float", [8, 16]
#mtype, n_estimators = "EfficientNetB0 binary", [8, 16]

filteredDF = df.loc[ df["model_type"] == mtype ]
#n_estimators = sorted(set(filteredDF["n_estimators"].dropna()))

for ne in n_estimators:
    dff = filteredDF.loc[ (filteredDF["n_estimators"] == ne) | (filteredDF["n_estimators"].isnull())]
    dff = dff.loc[ (dff["l_reg"] <= 0.5) | (dff["l_reg"].isnull())]

    #print(dff)
    #dff = dff.loc[ dff["l_Reg"] < 1000]
    #dff = df[ (df["model_type"] == mtype) and (df["n_estimators"] == ne or df["n_estimators"] == np.nan)]

    fig = px.bar(dff, x="nice_name",y="scores.accuracy_test",
        title="Test accuracy with T = {} and {} models".format(ne, mtype), 
        labels={
            "scores.accuracy_test": "Test accuracy",
            "nice_name": "Method"
        },
        text = "scores.accuracy_test"
    )
    fig.update_traces(textposition='outside')
    fig.update_layout(xaxis={'categoryorder':'category ascending'})
    fig.show()
    fig.write_image("images/{}_{}_{}.pdf".format("Accuracy",mtype.replace(" ", "-"), ne))

    fig = px.bar(dff, x="nice_name",y="scores.diversity_test",
        title="Test diversity with T = {} and {} models".format(ne, mtype), 
        labels={
            "scores.diversity_test": "Diversity",
            "nice_name": "Method"
        },
        log_y = True,
        text = "scores.diversity_test"
    )
    fig.update_traces(textposition='outside')
    fig.update_layout(xaxis={'categoryorder':'category ascending'})
    fig.show()
    fig.write_image("images/{}_{}_{}.pdf".format("Diversity",mtype.replace(" ", "-"), ne))

    fig = px.bar(dff, x="nice_name",y="scores.avg_accurcay_test",
        title="Average test accuracy with T = {} and {} models".format(ne, mtype), 
        labels={
            "scores.avg_accurcay_test": "Average test accuracy of each ensemble member",
            "nice_name": "Method"
        },
        text = "scores.avg_accurcay_test"
    )
    fig.update_traces(textposition='outside')
    fig.update_layout(xaxis={'categoryorder':'category ascending'})
    fig.show()
    fig.write_image("images/{}_{}_{}.pdf".format("AvgAccuracy",mtype.replace(" ", "-"), ne))

    fig = px.bar(dff, x="nice_name",y="complexity",
        title="Model complexity with T = {} and {} models".format(ne, mtype), 
        labels={
            "complexity": "Model complexity (# of trainable parameters)",
            "nice_name": "Method"
        },
        text = "complexity"
    )
    fig.update_traces(textposition='outside')
    fig.update_layout(xaxis={'categoryorder':'category ascending'})
    fig.show()
    fig.write_image("images/{}_{}_{}.pdf".format("Complexity",mtype.replace(" ", "-"), ne))

    fig = px.bar(dff, x="nice_name",y="scores.loss_train",
        title="Training loss with T = {} and {} models".format(ne, mtype), 
        labels={
            "scores.loss_train": "Training loss",
            "nice_name": "Method"
        },
        log_y = True,
        text = "scores.loss_train"
    )
    fig.update_traces(textposition='outside')
    fig.update_layout(xaxis={'categoryorder':'category ascending'})
    fig.show()
    fig.write_image("images/{}_{}_{}.pdf".format("Loss",mtype.replace(" ", "-"), ne))
        

